In [17]:
def writeWayPointFileSimple(cutPointList, fileName = "./2025TestCutPoint.gpx"):
    '''
    write cut point in to gpx file
    x, y to y, x ***
    input: list of [y,x]
    '''
    # print(content)
    file = open(fileName, "a")
    file.write("<?xml version='1.0' encoding='utf-8' ?>")
    file.write("\n")
    file.write('<gpx xmlns="http://www.topografix.com/GPX/1/1" xmlns:b="http://www.bizstation.jp/waypoint_extension" creator="Drogger GPS for Android 2.12.229" version="1.1">"')

    for point in cutPointList:
        writeContent = '<wpt lat="' + str(point[1]) + '" lon="' + str(point[0]) + '">'
        file.write("\n")
        file.write(writeContent)
        file.write("\n")
        file.write("</wpt>")
        # <wpt lat="38.255167122" lon="140.828141869">
        # </wpt>

    file.write("\n")
    file.write("</gpx>")
    file.close()

def getOriginalPoint(filePath = "./2025OriginalPoint.txt"):
    '''
    open original waypoint file got from gps (y,x)
    return as list [x,y] for calculation
    '''
    returnList = []
    with open(filePath, 'r') as file:
        content = file.read().split('\n')

    # split and invert, change to float from str
    for lines in content:
        lines = lines.split(",")
        returnList.append((float(lines[1]), float(lines[0])))

    return returnList


def calculateOffset():
    '''calculate offset from one of the 2024 data
    '''
    sampleA = (140.82826833165095, 38.255044085589624)
    sampleB = (140.82827608295284, 38.255042486768865)
    sampleC = (140.8282630158072, 38.25502569410004)
    sampleD = (140.8282707671091, 38.25502409527928)

    
    alpha = abs(sampleA[1] - sampleB[1])*1.1
    beta = abs(sampleA[0] - sampleB[0])/2
    gamma = abs(sampleB[1] - sampleD[1])*1.15
    sigma = abs(sampleB[0] - sampleD[0])*1.4

    # try offset
    originalPoint = (140.985897303, 37.363907403)
    FarOriginalPoint = (originalPoint[0] - gamma, originalPoint[1] + sigma)
    newA = (originalPoint[0] + alpha, originalPoint[1] + beta)
    newB = (originalPoint[0] - alpha, originalPoint[1] - beta)
    newC = (FarOriginalPoint[0] + alpha, FarOriginalPoint[1] + beta)
    newD = (FarOriginalPoint[0] - alpha, FarOriginalPoint[1] - beta)

    return alpha, beta, gamma, sigma

def calculateCutPoint(originalPathList,alpha, beta, gamma, sigma):
    '''
    calculate 4 cutpoint from original way point calculate with offsets
    '''
    returnList = []

    for originalPath in originalPathList:

        FarOriginalPoint = (originalPath[0] - gamma, originalPath[1] + sigma)
        newA = (originalPath[0] + alpha, originalPath[1] + beta)
        newB = (originalPath[0] - alpha, originalPath[1] - beta)
        newC = (FarOriginalPoint[0] + alpha, FarOriginalPoint[1] + beta)
        newD = (FarOriginalPoint[0] - alpha, FarOriginalPoint[1] - beta)

        returnList.append(newA)
        returnList.append(newB)
        returnList.append(newC)
        returnList.append(newD)

    return returnList


In [18]:
originalPathList = getOriginalPoint()
alpha, beta, gamma, sigma = calculateOffset()
cutPointList = calculateCutPoint(originalPathList, alpha, beta, gamma, sigma)
writeWayPointFileSimple(cutPointList)

